pour le demarage froid 


In [ ]:
!pip install kaggle

In [ ]:

from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"abdramdani","key":"0850d17ece69fcc4d3fdbf3d1fa789d4"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#data set api 
!kaggle datasets download -d ruchi798/bookcrossing-dataset

 96% 73.0M/76.1M [00:00<00:00, 74.9MB/s]
100% 76.1M/76.1M [00:01<00:00, 78.4MB/s]


In [ ]:
#code to extract bookcrossing-dataset.zip
from zipfile import ZipFile
file_name="bookcrossing-dataset.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print("done")


done


In [ ]:
#import
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt



In [ ]:
#download dataset (from dataset-finale.ipynb) or work on the kaggle dataset

In [ ]:
"""
books_data=pd.read_csv("/content/Books Data with Category Language and Summary/Preprocessed_data.csv",low_memory=False)
books_rat=pd.read_csv("/content/Book reviews/Book reviews/BX-Book-Ratings.csv",sep=';', encoding='latin-1',low_memory=False)
books_data=books_data.rename(columns={"isbn":"ISBN"})
print(books_data.shape,"\n",list(books_data.columns))
drop_c=['user_id','Unnamed: 0','age', 'location' , 'city', 'state', 'country',"rating"]
books_data=books_data.drop(drop_c,axis=1)
books_data=books_data.drop_duplicates(subset="ISBN")
books_data=books_data.drop_duplicates(subset="Summary")
#on enleve les column avec l'erreur de 9 
books_data=books_data[books_data["Summary"]!="9"]
books_data=books_data[books_data["Category"]!="9"]
#je garde que les livres en anglais 
books_data=books_data[books_data["Language"]=="en"]
print(books_data.shape)
"""


(1031175, 19) 
 ['Unnamed: 0', 'user_id', 'location', 'age', 'ISBN', 'rating', 'book_title', 'book_author', 'year_of_publication', 'publisher', 'img_s', 'img_m', 'img_l', 'Summary', 'Language', 'Category', 'city', 'state', 'country']
(129982, 11)


In [ ]:
"""
#jajoute le average rating et la somme des ratting pour les livres
#jenleve les rating =0 pour le calcule 
print("dataset des rating avant : ",books_rat.shape,books_rat["ISBN"].nunique(),books_rat["User-ID"].nunique())
books_rat_nnul=books_rat[books_rat["Book-Rating"]!=0]
print("dataset des rating apres l'elimination des valeurs ==0 : ",books_rat_nnul.shape,books_rat_nnul["ISBN"].nunique(),books_rat_nnul["User-ID"].nunique())
#nouveau diagramme


#average_rating
average_book=pd.DataFrame(books_rat_nnul.groupby("ISBN")["Book-Rating"].mean()).reset_index().rename(columns={"Book-Rating":"average_rating"})
#average_book.sort_values(by="average_rating",ascending=False)

books_data=pd.merge(books_data,average_book, on ="ISBN")

#somme rating
book_rating_Count=(books_rat_nnul.groupby(by=["ISBN"])["Book-Rating"].count().reset_index().
                   rename(columns={"Book-Rating":"SommeRating"}))
books_data=pd.merge(books_data,book_rating_Count, on ="ISBN")
print(books_data.shape)
#books_data
"""

dataset des rating avant :  (1149780, 3) 340556 105283
dataset des rating apres l'elimination des valeurs ==0 :  (433671, 3) 185973 77805
(75288, 13)


In [ ]:
"""
#j'utilise que les rating sans les 0
books_rating=pd.merge(books_rat_nnul,books_data["ISBN"],on="ISBN")
print(books_rating.shape,books_rating.columns)
"""

(216926, 3) Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object')


In [ ]:
"""
#on prend un user avec > 15 ratting 
print(books_rating.shape,books_rating["User-ID"].nunique(),books_rating["ISBN"].nunique())
#groupe1= books_rat_nnul["User-ID"].value_counts()
groupe1=pd.DataFrame(books_rating.groupby("User-ID")["ISBN"].count()).reset_index().rename(columns={"ISBN":"Nb_book_rated"})
print(groupe1.shape)
groupe1=groupe1.query("Nb_book_rated >=15")
print(groupe1.shape)
books_rat2=pd.merge(books_rating,groupe1["User-ID"], on ="User-ID",how="inner")
print(books_rat2.shape,books_rat2["User-ID"].nunique(),books_rat2["ISBN"].nunique())
"""

(216926, 3) 46162 75288
(46162, 2)
(2563, 2)
(118152, 3) 2563 55031


In [ ]:
"""
#je prends les livres qui existe dans books_rat2
books_rat_isbn=books_rat2.drop_duplicates(subset="ISBN")
print(books_rat_isbn.shape)
books_data2=pd.merge(books_data,books_rat_isbn["ISBN"], on ="ISBN",how="inner")
print(books_data2.shape,books_rat2["ISBN"].nunique())
"""

(55031, 3)
(55031, 13) 55031


In [ ]:
#sinon je telecharge de mon pc avec le pickle
#from pickle file
import pickle
pickle_in=open("/content/sample_data/books_data_final.pickle","rb")
books_data2=pickle.load(pickle_in)
pickle_in=open("/content/sample_data/books_rating_final.pickle","rb")
books_rat2=pickle.load(pickle_in)

In [ ]:
pickle_in=open("/content/sample_data/users_final.pickle","rb")
users=pickle.load(pickle_in)

In [ ]:
"""
#from csv file 
books_data2=pd.read_csv("/content/sample_data/books_data_final.csv")
books_rat2=pd.read_csv("/content/sample_data/books_rating_final.csv")
books_data2=books_data2.drop("Unnamed: 0",axis=1)
books_rat2=books_rat2.drop("Unnamed: 0",axis=1)
"""

'\n#from csv file \nbooks_data2=pd.read_csv("/content/sample_data/books_data_final.csv")\nbooks_rat2=pd.read_csv("/content/sample_data/books_rating_final.csv")\nbooks_data2=books_data2.drop("Unnamed: 0",axis=1)\nbooks_rat2=books_rat2.drop("Unnamed: 0",axis=1)\n'

In [ ]:
#je recupere les category de l'utilisateur + la liste des category de cette utilisateur 

x=np.random.randint(2563)
user_test=users.iloc[x]
user_rat=books_rat2[books_rat2["User-ID"]==user_test["User-ID"]]
user_rat

,User-ID,ISBN,Book-Rating
116635,233041,0373441800,4
116636,233041,051513449X,10
116637,233041,0515115630,10
116638,233041,0373764731,3
116639,233041,0373255632,10
116640,233041,0553446398,8
116641,233041,0373240252,7
116642,233041,0373271506,3
116643,233041,0373030916,6
116644,233041,0373032188,5


In [ ]:
#on recupere la category
cat_user=["['History']"]
zzz=books_data2[books_data2["Category"].isin(cat_user)]

In [ ]:
average_all=zzz["average_rating"].mean()
nb_vote_min=zzz["SommeRating"].quantile(0.99)
print(nb_vote_min)
print(average_all)

114.53999999999998
8.089482780704925


In [ ]:
def weighted_rating(data,min=nb_vote_min,aver=average_all):
  ar=data["average_rating"]
  vc=data["SommeRating"]
  return (vc/(vc+min)*ar) + (min/(min+vc)*aver)

In [ ]:
#japplique la formule sur chaque column
zzz["weighted_score"]=zzz.apply(weighted_rating,axis=1)
zzz=zzz.sort_values("weighted_score",ascending=False)
zzz

In [ ]:
recommandation=zzz.merge(user_rat["ISBN"],how="left",on="ISBN")


In [ ]:
#je retourne la liste de recommandation dans ce cas la 
recommandation

,ISBN,book_title,book_author,year_of_publication,publisher,img_s,img_m,img_l,Summary,Language,Category,average_rating,SommeRating,weighted_score
0,0449005615,Seabiscuit: An American Legend,LAURA HILLENBRAND,2002.0,Ballantine Books,http://images.amazon.com/images/P/0449005615.0...,http://images.amazon.com/images/P/0449005615.0...,http://images.amazon.com/images/P/0449005615.0...,True story of how three men and a great raceho...,en,['History'],8.483333,120,8.290992
1,0609608444,"The Devil in the White City : Murder, Magic, a...",ERIK LARSON,2003.0,Crown,http://images.amazon.com/images/P/0609608444.0...,http://images.amazon.com/images/P/0609608444.0...,http://images.amazon.com/images/P/0609608444.0...,An account of the Chicago World&#39;s Fair of ...,en,['History'],8.656250,32,8.213248
2,0679721037,Hiroshima,John Hersey,1989.0,Vintage Books USA,http://images.amazon.com/images/P/0679721037.0...,http://images.amazon.com/images/P/0679721037.0...,http://images.amazon.com/images/P/0679721037.0...,The classic tale of the day the first atom bom...,en,['History'],8.692308,26,8.201006
3,0393317552,"Guns, Germs, and Steel: The Fates of Human Soc...",Jared Diamond,1999.0,W. W. Norton & Company,http://images.amazon.com/images/P/0393317552.0...,http://images.amazon.com/images/P/0393317552.0...,http://images.amazon.com/images/P/0393317552.0...,Traces the development of primitive societies ...,en,['History'],8.416667,36,8.167725
4,067973743X,Bury Me Standing: The Gypsies and Their Journe...,Isabel Fonseca,1996.0,Vintage Books USA,http://images.amazon.com/images/P/067973743X.0...,http://images.amazon.com/images/P/067973743X.0...,http://images.amazon.com/images/P/067973743X.0...,A revealing portrait of a disappearing culture...,en,['History'],8.909091,11,8.161298
5,014011369X,"And the Band Played on: Politics, People, and ...",Randy Shilts,1995.0,Penguin Books,http://images.amazon.com/images/P/014011369X.0...,http://images.amazon.com/images/P/014011369X.0...,http://images.amazon.com/images/P/014011369X.0...,An examination of the AIDS crisis exposes the ...,en,['History'],9.125000,8,8.157086
6,0743410068,War Letters : Extraordinary Correspondence fro...,Andrew Carroll,2002.0,Washington Square Press,http://images.amazon.com/images/P/0743410068.0...,http://images.amazon.com/images/P/0743410068.0...,http://images.amazon.com/images/P/0743410068.0...,"Here are letters from the Civil War, World War...",en,['History'],9.333333,6,8.151397
7,0345465083,Seabiscuit,LAURA HILLENBRAND,2003.0,Ballantine Books,http://images.amazon.com/images/P/0345465083.0...,http://images.amazon.com/images/P/0345465083.0...,http://images.amazon.com/images/P/0345465083.0...,"Retraces the journey of Seabiscuit, a horse wi...",en,['History'],8.241935,62,8.143023
8,0060175869,From Dawn to Decadence: 1500 to the Present: 5...,Jacques Barzun,2000.0,HarperCollins,http://images.amazon.com/images/P/0060175869.0...,http://images.amazon.com/images/P/0060175869.0...,http://images.amazon.com/images/P/0060175869.0...,Only after a lifetime of separate studies cove...,en,['History'],9.000000,7,8.141923
9,0449204553,A Walk Across America,Peter Jenkins,1989.0,Fawcett Books,http://images.amazon.com/images/P/0449204553.0...,http://images.amazon.com/images/P/0449204553.0...,http://images.amazon.com/images/P/0449204553.0...,This book chronicles the author&#39;s journey ...,en,['History'],8.777778,9,8.139626
